# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_cs
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
   )


#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify_cs')


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# session_id AND item_in_session is the clustering column


session.execute("""
    CREATE TABLE IF NOT EXISTS artists (
    session_id int,
    item_in_session int,
    artist text,
    song_title text,
    duration float,
    history text,
    PRIMARY KEY (session_id, item_in_session)
    )
""")


            
    

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

# artist
# firstName of user
# gender of user
# item number in session
# last name of user
# length of the song
# level (paid or free song)
# location of the user
# sessionId
# song title
# userId

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artists(session_id, item_in_session, artist, song_title, duration)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"

        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
print(query_1_result)

In [13]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query_1 = """
    SELECT artist, song_title, duration FROM artists WHERE session_id = 338 AND item_in_session = 4
"""

for row in query_1_result:
    print(row.artist, row.song_title, row.duration)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

#### Query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [21]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

session.execute("""
    CREATE TABLE IF NOT EXISTS users_log (
    user_id int,
    session_id int,
    item_in_session int,
    artist text,
    song text,
    user_first_name text,
    user_last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session)
    )
""")
                    

In [22]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_log (user_id, session_id, item_in_session, artist, song, user_first_name, user_last_name) \
        VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


In [27]:

#2. Give me only the following: name of artist, song (sorted by itemInSession) 
# and user (first and last name) for userid = 10, sessionid = 182
    

query_2 = """SELECT artist, song, user_first_name, user_last_name FROM users_log
WHERE user_id = 10 AND session_id = 182
"""

query_results_2 = session.execute(query_2)
for row in query_results_2:
    print ( row.artist, row.song, row.user_first_name, row.user_last_name)
    
    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [41]:
# check of results
check = pd.read_csv(file)
mask = (check['sessionId'] == 182) & (check['userId'] == 10)
check.loc[mask].sort_values(by = ['userId','sessionId','itemInSession'], ascending = [True,True,True]) \
[['artist','song','itemInSession','firstName','lastName']]
# from the result we can see that the entries were corretly ordered based on clusterung column itemInSession.

,artist,song,itemInSession,firstName,lastName
4704,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
4705,Three Drives,Greece 2000,1,Sylvie,Cruz
4706,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
4707,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


In [42]:
# The result looks correct as the one I get with the CQL.

#### Query 3 
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [44]:
# Give me every user name (first and last) in my music app history
# who listened to the song 'All Hands Against His Own'

session.execute("""
    CREATE TABLE IF NOT EXISTS histories_log (
    song text,
    user_id int,
    user_first_name text,
    user_last_name text,
    PRIMARY KEY (song, user_id)
    )
""")


In [34]:
# artist
# firstName of user
# gender of user
# item number in session
# last name of user
# length of the song
# level (paid or free song)
# location of the user
# sessionId
# song title
# userId

In [45]:
with open (file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO histories_log (song, user_id, user_first_name, user_last_name) \
        VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [46]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query_3 = """
SELECT user_first_name, user_last_name, song FROM histories_log WHERE (song = 'All Hands Against His Own')
"""
query_3_result = session.execute(query_3)
for row in query_3_result:
    print( row.song, row.user_first_name, row.user_last_name)
                    

All Hands Against His Own Jacqueline Lynch
All Hands Against His Own Tegan Levine
All Hands Against His Own Sara Johnson


In [47]:
# Before I was just unsing the song as the primary key, 
# but that lead to the fact that all the users have been overwritten and
# I just got back 1 row. 
# So I set as the clustering column, user_id to avoid this error.


In [40]:
# check of results
check = pd.read_csv(file)
mask = (check['song'] == 'All Hands Against His Own')
check.loc[mask].sort_values(by = 'userId', ascending = True)[['firstName','lastName','userId']]

,firstName,lastName,userId
6298,Jacqueline,Lynch,29
2792,Tegan,Levine,80
5135,Sara,Johnson,95


In [ ]:
# Answer does look correct 

### Drop the tables before closing out the sessions

In [48]:
query = "DROP TABLE IF EXISTS artists"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS users_log"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS histories_log"
try:
    session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [49]:
session.shutdown()
cluster.shutdown()